This code is for EN5423 class at GIST, Republic of Korea, and created by Dr. Hyunglok Kim.  
**Contact information**: hyunglokkim@gist.ac.kr  
**License**: This work is licensed for non-commercial use only.  
**Restrictions**: Do not use this material without permission for teaching or developing other classes.

In [64]:
import numpy as np
import pandas as pd
from scipy.stats import binomtest, binom_test, norm
from scipy.stats import wilcoxon
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import permutation_test
from scipy.stats import rankdata

In [7]:
Above = np.array([12, 15, 11, 41, 106, 63, 296, 53, 20, 110, 429, 185])
Below = np.array([9, 8, 38, 24, 48, 17, 11, 41, 14, 60, 53, 124])
signdiff = np.sign(Above - Below)
nymph_list = pd.DataFrame({'Above': Above, 'Below': Below, 'signdiff': signdiff})
nymph_list

,Above,Below,signdiff
0,12,9,1
1,15,8,1
2,11,38,-1
3,41,24,1
4,106,48,1
5,63,17,1
6,296,11,1
7,53,41,1
8,20,14,1
9,110,60,1


In [8]:
p_value = binomtest(11, 12, alternative='greater')
p_value

BinomTestResult(k=11, n=12, alternative='greater', statistic=0.9166666666666666, pvalue=0.003173828125)

In [16]:
# Example 1. Mayfly nymphs—Exact sign test, small samples.
def binomial_test_results(successes, trials, alternative='greater', alpha=0.05):
    p_value = binom_test(successes, trials, alternative=alternative)
    sample_estimate = successes / trials
    
    # Calculate Wilson score interval
    z_critical = np.abs(np.round(norm.ppf(alpha / 2), 4))
    p_hat = successes / trials
    interval_half_width = z_critical * np.sqrt((p_hat * (1 - p_hat)) / trials + (z_critical**2) / (4 * trials**2))
    lower_bound = max(0, p_hat - interval_half_width)
    upper_bound = min(1, p_hat + interval_half_width)
    
    results = f"Exact binomial test\n"
    results += f"data: {successes} and {trials}\n"
    results += f"number of successes = {successes}, number of trials = {trials}, p-value = {p_value:.6f}\n"
    results += f"alternative hypothesis: true probability of success is {alternative} than 0.5\n"
    results += f"95 percent confidence interval:\n"
    results += f"{lower_bound:.7f} {upper_bound:.7f}\n"
    results += f"sample estimates:\n"
    results += f"probability of success\n"
    results += f"{sample_estimate:.7f}"
    
    return results

# Example usage:
results = binomial_test_results(11, 12, alternative='greater')
print(results)


Exact binomial test
data: 11 and 12
number of successes = 11, number of trials = 12, p-value = 0.003174
alternative hypothesis: true probability of success is greater than 0.5
95 percent confidence interval:
0.6928901 1.0000000
sample estimates:
probability of success
0.9166667


/var/folders/_4/qjq3yrjn047588zvn9_fc2gc0000gn/T/ipykernel_33926/2680924518.py:3: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_value = binom_test(successes, trials, alternative=alternative)


In [17]:
# Example 2. Mayfly nymphs—Large-sample approximation for the sign test.
def prop_test_results(successes, trials, alternative='larger', alpha=0.05):
    # Calculate the sample proportion
    p_hat = successes / trials
    
    # Perform the one-sample proportions z-test
    z_stat, p_value = proportions_ztest(successes, trials, value=0.5, alternative=alternative)
    
    # Calculate the confidence interval using the normal approximation
    z_critical = np.round(norm.ppf(1 - alpha / 2), 7)
    interval_half_width = z_critical * np.sqrt(p_hat * (1 - p_hat) / trials)
    lower_bound = max(0, p_hat - interval_half_width)
    upper_bound = min(1, p_hat + interval_half_width)
    
    # Construct the results string
    results = f"1-sample proportions test with continuity correction\n"
    results += f"data: {successes} out of {trials}, null probability 0.5\n"
    results += f"X-squared = {z_stat:.2f}, df = 1, p-value = {p_value:.6f}\n"
    results += f"alternative hypothesis: true p is greater than 0.5\n"
    results += f"95 percent confidence interval:\n"
    results += f"{lower_bound:.7f} {upper_bound:.7f}\n"
    results += f"sample estimates:\n"
    results += f"p = {p_hat:.7f}"
    
    return results

# Example usage:
results = prop_test_results(11, 12, alternative='larger')
print(results)


1-sample proportions test with continuity correction
data: 11 out of 12, null probability 0.5
X-squared = 5.22, df = 1, p-value = 0.000000
alternative hypothesis: true p is greater than 0.5
95 percent confidence interval:
0.7602898 1.0000000
sample estimates:
p = 0.9166667


In [18]:
def sign_test_approximation(S_plus, n):
    # Calculate mean and standard deviation of S_plus
    mu_S_plus = n / 2
    sigma_S_plus = 0.5 * np.sqrt(n)
    
    # Calculate the test statistic Z_plus
    if S_plus > mu_S_plus:
        Z_plus = (S_plus - mu_S_plus - 0.5) / sigma_S_plus
    elif S_plus < mu_S_plus:
        Z_plus = (S_plus - mu_S_plus + 0.5) / sigma_S_plus
    else:
        Z_plus = 0
    
    return Z_plus

def sign_test_p_value(Z_plus):
    # Calculate the p-value using the standard normal distribution
    p_value = 1 - norm.cdf(Z_plus)
    
    return p_value

def sign_test_results(successes, trials, alpha=0.05):
    n = trials
    S_plus = successes
    
    # Calculate test statistic Z_plus
    Z_plus = sign_test_approximation(S_plus, n)
    
    # Calculate p-value
    p_value = sign_test_p_value(Z_plus)
    
    # Construct results string
    results = f"1-sample proportions test with continuity correction\n"
    results += f"data: {successes} out of {trials}, null probability 0.5\n"
    results += f"X-squared = {Z_plus**2:.2f}, df = 1, p-value = {p_value:.6f}\n"
    results += f"alternative hypothesis: true p is greater than 0.5\n"
    results += f"95 percent confidence interval:\n"
    results += f"sample estimates:\n"
    results += f"p = {successes/trials:.7f}"
    
    return results

# Example usage:
results = sign_test_results(11, 12)
print(results)


1-sample proportions test with continuity correction
data: 11 out of 12, null probability 0.5
X-squared = 6.75, df = 1, p-value = 0.004687
alternative hypothesis: true p is greater than 0.5
95 percent confidence interval:
sample estimates:
p = 0.9166667


In [19]:
# Example 3. Mayfly nymphs—Exact signed-rank test.
def wilcoxon_signed_rank_test(Above, Below, alternative='greater'):
    # Create DataFrame
    nymph_sr = pd.DataFrame({'Above': Above, 'Below': Below})
    
    # Calculate the differences
    nymph_sr['D.i'] = nymph_sr['Above'] - nymph_sr['Below']
    
    # Calculate the absolute differences
    nymph_sr['SR.i'] = abs(nymph_sr['D.i'])
    
    # Calculate the signed ranks
    nymph_sr['SR.i'] = nymph_sr['SR.i'].rank() * nymph_sr['D.i'] / abs(nymph_sr['D.i'])
    
    # Perform Wilcoxon signed-rank test
    statistic, p_value = wilcoxon(Above, Below, alternative=alternative, zero_method='pratt', correction=True)
    
    # Output the results
    
    print("V =", statistic, ", p-value =", p_value)
    print("alternative hypothesis: true location shift is", alternative, "than 0")
    
    return nymph_sr[['Above', 'Below', 'D.i', 'SR.i']]

# Example usage:
Above = [12, 15, 11, 41, 106, 63, 296, 53, 20, 110, 429, 185]
Below = [9, 8, 38, 24, 48, 17, 11, 41, 14, 60, 53, 124]

nymph_sr = wilcoxon_signed_rank_test(Above, Below, alternative='greater')
nymph_sr

V = 72.0 , p-value = 0.00341796875
alternative hypothesis: true location shift is greater than 0


,Above,Below,D.i,SR.i
0,12,9,3,1.0
1,15,8,7,3.0
2,11,38,-27,-6.0
3,41,24,17,5.0
4,106,48,58,9.0
5,63,17,46,7.0
6,296,11,285,11.0
7,53,41,12,4.0
8,20,14,6,2.0
9,110,60,50,8.0


In [21]:
differences = Above - Below

# Check the number of observations
n = len(differences)

# Choose the method based on the sample size
if n < 20:
    method = 'exact'
else:
    method = 'approx'

# Perform the Wilcoxon signed-rank test with a specified method
stat, p_value = wilcoxon(differences, alternative='greater', method=method)
print("Wilcoxon signed-rank test")
print(f"Test Statistic: {stat}, P-value: {p_value}, Method used: {method}")

method = 'approx'
# Perform the Wilcoxon signed-rank test with a specified method
stat, p_value = wilcoxon(differences, alternative='greater', method=method)
print("Wilcoxon signed-rank test")
print(f"Test Statistic: {stat}, P-value: {p_value}, Method used: {method}")

Wilcoxon signed-rank test
Test Statistic: 72.0, P-value: 0.00341796875, Method used: exact
Wilcoxon signed-rank test
Test Statistic: 72.0, P-value: 0.004816487886294337, Method used: approx


In [63]:
# Example 5. Mayfly nymphs—Permutation version of the signed-rank test.
def wilcoxon_signed_rank_test(above, below, alternative='greater', n_resamples=1000000000):
    # Calculate the differences and filter out zeros
    differences = np.array(above) - np.array(below)
    non_zero_differences = differences[differences != 0]

    # Calculate ranks of the absolute differences
    ranks = rankdata(np.abs(non_zero_differences))

    # Compute signed ranks
    signed_ranks = ranks * np.sign(non_zero_differences)

    # Compute the sum of positive ranks for the actual data
    w_plus = np.sum(signed_ranks[signed_ranks > 0])
    
    # Number of all possible permutations
    total_permutations = 2 ** len(non_zero_differences)
    num_combinations = min(n_resamples, total_permutations)

    # Collect permutation stats
    permutation_stats = []
    for _ in range(num_combinations):
        # Randomly flip signs
        random_signs = np.random.choice([-1, 1], size=len(non_zero_differences))
        permuted_signed_ranks = signed_ranks * random_signs
        permuted_stat = np.sum(permuted_signed_ranks[permuted_signed_ranks > 0])
        permutation_stats.append(permuted_stat)

    # Calculate p-value
    if alternative == 'greater':
        p_value = np.mean([stat >= w_plus for stat in permutation_stats])
    elif alternative == 'less':
        p_value = np.mean([stat <= w_plus for stat in permutation_stats])
    else:  # two-sided
        p_value = np.mean([np.abs(stat) >= np.abs(w_plus) for stat in permutation_stats])

    return w_plus, p_value

# Example data
Above = [12, 15, 11, 41, 106, 63, 296, 53, 20, 110, 429, 185]
Below = [9, 8, 38, 24, 48, 17, 11, 41, 14, 60, 53, 124]

# Running the test
stat, p_val = wilcoxon_signed_rank_test(Above, Below, alternative='greater')
print(f"Test Statistic: {stat}, P-value: {p_val}")

Test Statistic: 72.0, P-value: 0.00341796875


In [66]:
# Example 6. Mayfly nymphs—Signed-rank test on logarithms.

differences = np.log(Above) - np.log(Below)
# Perform the Wilcoxon signed-rank test with a specified method
method = 'exact'
stat, p_value = wilcoxon(differences, alternative='greater', method=method)
print("Wilcoxon signed-rank test")
print(f"Test Statistic: {stat}, P-value: {p_value}, Method used: {method}")

Wilcoxon signed-rank test
Test Statistic: 69.0, P-value: 0.008056640625, Method used: exact
